# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
clients = pd.read_csv('/datasets/data.csv')
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 колонок, стоит обратить внимание на колонки days employed и total_income, в них количество значение non-null отличается, значит, в данных колонках есть пропущенные значения, которые далее необходимо обработать. Также тип данных float в колонке days employed стоит пересмотреть и преобразовать к целые числа

In [2]:
clients.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [3]:
clients.duplicated().sum()

54

## Шаг 2. Предобработка данных

In [4]:
#Первыми в глаза бросаются отработанные дни, необходимо избавиться от отрицательных значений, применив метод абс и вернув тем самым абсолютные значения чиссел
#Также слишком большие значения записаны в часах, следовательно разделим на 24, чтобы получить приблизтельное количество дней 
clients['days_employed'] = clients['days_employed'].apply(abs)
clients.loc[clients['days_employed'] > 30000, 'days_employed'] = clients.loc[clients['days_employed'] > 30000, 'days_employed'] / 24
#Пропуски в данном столбце можно заменить на среднее количество отработанных дней. Пропуски случайные, если посмотреть на образование и источник дохода, столько человек не могло не работать ни одного дня. Количество дней также не должно повлиять на выводы по поставленным задачам
days_avg = clients['days_employed'].mean()
clients['days_employed'] = clients['days_employed'].fillna(days_avg)
display(clients.head(20))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [5]:
#Перейдем к столбцу с количеством детей, найдем уникальные значения
clients['children'].unique()


array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [6]:
#20 детей, скорее всего, ошибка и ноль был поставлен случайно вместо знака препинания, и отрицательное значение приведем к нулю
#Для удобства будущего анализа, возможно, нам понадобится столбец с категорией "есть или нет дети"
clients.loc[clients['children'] == 20, 'children'] = 2
clients.loc[clients['children'] == -1, 'children'] = 0
def children_id(children):
    if children == 0:
        return '0'    
    return '1'
clients['children_id'] = clients['children'].apply(children_id)
display(clients['children_id'].head(15))

0     1
1     1
2     0
3     1
4     0
5     0
6     0
7     0
8     1
9     0
10    1
11    0
12    0
13    0
14    0
Name: children_id, dtype: object

In [7]:

clients['dob_years'].unique() 

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [8]:
#проверяем процент пропусков 
display(clients[clients['dob_years'] == 0].count() / (clients['dob_years'].count()))
#процент пропусков от общего составляет 0,47, поэтому можно удалить их без значительного влияния на дальнейший анализ
clients[clients['dob_years'] == 0].dropna()


children            0.004692
days_employed       0.004692
dob_years           0.004692
education           0.004692
education_id        0.004692
family_status       0.004692
family_status_id    0.004692
gender              0.004692
income_type         0.004692
debt                0.004692
total_income        0.004228
purpose             0.004692
children_id         0.004692
dtype: float64

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_id
99,0,14439.234121,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль,0
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем,0
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью,1
578,0,16577.356876,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости,0
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19371,1,2356.250615,0,высшее,0,женат / замужем,0,F,компаньон,0,77936.218285,автомобиль,1
20462,0,14113.952856,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья,0
20577,0,13822.552977,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость,0
21179,2,108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости,1


In [9]:
clients['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Колонка с образованием имеет разный регистр, необходимо привести все к нижнему регистру для правильности дальнейших подсчетов и по правилам стилистики. 

In [10]:
clients['education'] = clients['education'].str.lower()
clients['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [11]:
clients['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [12]:
#по аналогии с образованием обрабатываем колонку семейного положения 
clients['family_status'] = clients['family_status'].str.lower()

In [13]:
#Переведем в целые числа дни стажа, для проверки себя вывела на экран результат
clients['days_employed'] = clients['days_employed'].astype('int64')
display(clients['days_employed'].head(10))

0     8437
1     4024
2     5623
3     4124
4    14177
5      926
6     2879
7      152
8     6929
9     2188
Name: days_employed, dtype: int64

Возраст, стаж, дети, образование и семейное положение приведены в доступный для анализа вид, сейчас осталось удалить явные дубликаты методом drop_duplicates и еще раз посмотрим информацию о таблице, проверим еще раз пропуски и количество дубликатов

In [14]:
#Так же, как и колонку со стажем работы, обработаем пропущенные значение в доходе и заполним пропуски средним доходом
#Существует множество источников дохода, поэтому средний доход будет оптимальным решением 
income_avg = clients['total_income'].mean()
clients['total_income'] = clients['total_income'].fillna(income_avg)
clients['total_income'] = clients['total_income'].astype('int64')

In [15]:
clients.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
children_id         0
dtype: int64

In [16]:
clients.duplicated().sum()

71

In [17]:
clients = clients.drop_duplicates().reset_index(drop=True)

In [18]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
children_id         21454 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


Первым шагом была работа над отработанными днями, мы избавились от отрицательных значений, слишком большие значения разделили на 24, и преобразовали значения в целые числа. Также на этом шаге мы привели к нижнему регистру данные, создали отдельную колонку по наличию детей. Заменили пропущенные значения в стаже и доходе. По итогу у нас не осталось дубликатов и пропущенных значений.

### Лемматизация

In [19]:
#Для лемматизации возьмем столбец с указанием цели кредита, далее подсчитаем количество слов
from pymystem3 import Mystem
m = Mystem()
from collections import Counter
text = clients['purpose'].unique()
purpose_one = ','.join(text)
lemmas = m.lemmatize(purpose_one)
Counter(lemmas)


Counter({'покупка': 10,
         ' ': 59,
         'жилье': 7,
         ',': 37,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

In [20]:
#Напишем функцию для категоризации и создадим отдельный столбец с категорией цели кредита, по количеству слов выше взяли четыре большие группы целей кредита и группу прочих целей
def category(row):
    purpose = row['purpose']
    purpose_lemmas = m.lemmatize(purpose)
    if 'жилье' in purpose_lemmas or 'недвижимость' in purpose_lemmas:
        return 'операции с недвижимостью'
    elif 'автомобиль' in purpose_lemmas:
        return 'покупка автомобиля' 
    elif 'образование' in purpose_lemmas:
        return 'получение образования'
    elif 'свадьба' in purpose_lemmas:
        return 'свадьба'
    return 'прочие цели'

clients['purpose_lemmas'] = clients.apply(category, axis = 1)
display(clients['purpose_lemmas'].head(15))

0     операции с недвижимостью
1           покупка автомобиля
2     операции с недвижимостью
3        получение образования
4                      свадьба
5     операции с недвижимостью
6     операции с недвижимостью
7        получение образования
8                      свадьба
9     операции с недвижимостью
10    операции с недвижимостью
11    операции с недвижимостью
12                     свадьба
13          покупка автомобиля
14    операции с недвижимостью
Name: purpose_lemmas, dtype: object

In [21]:
clients['income_type'].drop_duplicates()
#проверка столбца 

0              сотрудник
4              пенсионер
5              компаньон
26           госслужащий
3132         безработный
5931     предприниматель
9399             студент
20778          в декрете
Name: income_type, dtype: object

In [22]:
#Переходим к колонке дохода, по аналогии с отработанными днями следует преобразовать в целые числа, 
# а также рассортировать доход на три типичные группы низкого, среднего и высокого уровня дохода. 
clients['total_income'].median()

def income_category(income):
    if income <= 60000:
        return 'низкий уровень'
    elif income <= 120000:
        return 'средний уровень'
    return 'высокий уровень'
clients['income_category'] = clients['total_income'].apply(income_category)
display(clients.head(15))





,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_id,purpose_lemmas,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,операции с недвижимостью,высокий уровень
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1,покупка автомобиля,средний уровень
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0,операции с недвижимостью,высокий уровень
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,1,получение образования,высокий уровень
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,0,свадьба,высокий уровень
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,0,операции с недвижимостью,высокий уровень
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,0,операции с недвижимостью,высокий уровень
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,0,получение образования,высокий уровень
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,1,свадьба,средний уровень
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,0,операции с недвижимостью,высокий уровень


Таблица готова к анализу, весь текст приведен к нижнему регистру, созданы столбы с категориями по некоторым позициям, некоторые значения преобразованы в целые числа.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [23]:
children_rep = clients.pivot_table(index = 'children_id', values = 'debt', aggfunc = ['sum','count', 'mean'])
children_rep.columns= ['debt','total', '%']
children_rep['%'] = (children_rep['%'] * 100).round(1)
children_rep = children_rep.sort_values(by=['total'], ascending=False)
children_rep


,debt,total,%
children_id,,,
0,1064,14138,7.5
1,677,7316,9.3


In [24]:
children_rep2 = clients.pivot_table(index = 'children', values = 'debt', aggfunc = ['sum','count', 'mean'])
children_rep2.columns= ['debt','total', '%']
children_rep2['%'] = (children_rep2['%'] * 100).round(1)
children_rep2 = children_rep2.sort_values(by=['total'], ascending=False)
children_rep2


,debt,total,%
children,,,
0,1064,14138,7.5
1,444,4808,9.2
2,202,2128,9.5
3,27,330,8.2
4,4,41,9.8
5,0,9,0.0


Разница между возвратом кредита в срок и наличием детей составляет всего около 2%, на основе чего можно сделать вывод о том, что наличие детей слабо влияет на сроки кредита в срок

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [25]:
family_rep = clients.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['sum','count', 'mean'])
family_rep.columns= ['debt','total', '%']
family_rep['%'] = (family_rep['%'] * 100).round(1)
family_rep = family_rep.sort_values(by=['total'], ascending=False)
family_rep


,debt,total,%
family_status,,,
женат / замужем,931,12339,7.5
гражданский брак,388,4151,9.3
не женат / не замужем,274,2810,9.8
в разводе,85,1195,7.1
вдовец / вдова,63,959,6.6


Самый высокий процент должников в графе "не женат / не замужем".
Самый низкий процент должников у людей с семейным положением "вдовец / вдова".
Разница между самым высоким и самым низким составляет 3%. При этом процент людей в гражданском браке близок к проценту в графе "не женат / не замужем", можно сделать вывод что семейное положение слабо влияет на возможность выплаты кредита в срок

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
income_rep = clients.pivot_table(index = 'income_category', values = 'debt', aggfunc = ['sum','count', 'mean'])
income_rep.columns= ['debt','total', '%']
income_rep['%'] = (income_rep['%'] * 100).round(1)
income_rep = income_rep.sort_values(by=['total'], ascending=False)
income_rep


,debt,total,%
income_category,,,
высокий уровень,1190,14609,8.1
средний уровень,502,6039,8.3
низкий уровень,49,806,6.1


Согласно процентной разницы между уровнями доходов, уровень дохода не влияет на срок выплаты кредита

- Как разные цели кредита влияют на его возврат в срок?

In [27]:
purpose_rep = clients.pivot_table(index = 'purpose_lemmas', values = 'debt', aggfunc = ['sum','count', 'mean'])
purpose_rep.columns= ['debt','total', '%']
purpose_rep['%'] = (purpose_rep['%'] * 100).round(1)
purpose_rep = purpose_rep.sort_values(by=['total'], ascending=False)
purpose_rep


,debt,total,%
purpose_lemmas,,,
операции с недвижимостью,782,10811,7.2
покупка автомобиля,403,4306,9.4
получение образования,370,4013,9.2
свадьба,186,2324,8.0


Самый низкий процент должников по кредиту относится к кредитам, взятым для совершения операций с недвижимостью. 
Однаков, разница процентов по целям кредита также небольшая, поэтому можно сделать вывод, что цель кредита имеет небольшое влияние на возврат кредита в срок.

Получив файл с данным, в первую очередь мы изучили общую информацию.
Был произведен первичный подсчет дубликатов и поиск пропущенных значений. Далее мы приступили к предобработке данных.
1. В колонке days_employed мы преобразовали отрицательные значения, так как данные предоставлены в днях, то слишком большие значения мы разлелили на 24. Также в этой колонке были пропущенные значние, которые мы заменили на среднее количсетво отработанных дней. Кроме того, тип данных был заменен на int64.
2. Была создана отдельная колонка для обозначения наличия детей.
3. Текстовые значения в таблице были приведены к нижнему регистру.
4. Пропущенные значения в колонке total_income были заменены на среднее значение, также тип данных был изменен на int. Была создана колонка, в значения из total_income рассортированы на три группы: низкий уровень дохода, средний уровень и высокий уровень.
5. При помощи лемматизации мы обработали колонку с указанием цели кредита, также была создана новая колонка с категориями цели кредита.
На основе проведенного анализа были сделаны следующие выводы:
Разница между возвратом кредита в срок и наличием детей составляет всего около 2%, на основе чего можно сделать вывод о том, что наличие детей слабо влияет на сроки кредита в срок. Разница между количеством детей по критерию возврата в срок также составляет 2-3%. Из этого следует, что для более точного анализа лучше брать количество детей, а не только факт наличия.
Самый высокий процент должников в графе "не женат / не замужем". Самый низкий процент должников у людей с семейным положением "вдовец / вдова". Разница между самым высоким и самым низким составляет 3%. При этом процент людей в гражданском браке близок к проценту в графе "не женат / не замужем", можно сделать вывод что семейное положение слабо влияет на возможность выплаты кредита в срок.
У людей с низким уровнем дохода ниже процент, разница в процентах с людьми с высоким уровнем составляет 2%, по итогу люди с низким доходом имеют меньше долгов. 
Самый низкий процент должников по кредиту относится к кредитам, взятым для совершения операций с недвижимостью. 
Однаков, разница процентов по целям кредита также небольшая, поэтому можно сделать вывод, что цель кредита имеет небольшое влияние на возврат кредита в срок.

Делая вывод, можно сказать, что наличие детей, семейное положение, уровень дохода и цель влияют на возврат кредита в срок. Судя по данным, цели кредита люди писали сами, для более точной сортировки, например разделения на "коммерческую недвижимость" и "жилую" можно в анкету добавить графы с выбором нужной цели, а также строчку для "прочих целей".
